In [215]:
# Блок 1: Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, ShuffleSplit
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler

In [216]:
# Блок 2: Загрузка данных
data = pd.read_excel('ун 90-10.xlsx')
df = pd.DataFrame(data)

print(df.columns.tolist())

['Yн', 'х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']


In [217]:
# Блок 3: Определение входных и выходных данных
X = df[['х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']]
y = df['Yн']

In [218]:
# Блок 4: Разделение данных на обучающую, тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=73)

In [219]:
from sklearn.compose import ColumnTransformer

# приведение всех признаков к одному масштабу
scaler =  StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Обучение и преобразование обучающего набора данных
X_test_scaled = scaler.transform(X_test)        # Преобразование тестового набора

In [220]:
# Блок 5: Сетка гиперпараметров и кросс-валидация
# Задание сетки гиперпараметров для модели MLPRegressor
# Определение параметров для поиска по сетке
param_grid = {
    'hidden_layer_sizes': [(8,), (10,), (12,)],
    'activation': ['logistic'],
    'solver': ['adam'],
    'alpha': [0.92, 0.96, 1.0],
    'beta_1': [0.8, 0.85, 0.9],
    'beta_2': [0.9, 0.95, 0.99],
    'early_stopping': [True],
    'learning_rate_init': [0.03, 0.04, 0.05],
    'tol': [0.01, 0.05, 0.1],
    'epsilon': [1e-6],
    'max_iter': [200],
    'n_iter_no_change': [5, 10],
    'validation_fraction': [0.1, 0.2, 0.3]
}


# Настройка кросс-валидации
cv = ShuffleSplit(
    n_splits=5,         # количество фолдов
    test_size=0.1,       # перемешивание данных
    random_state=42     # для воспроизводимости результатов
)

grid_search = GridSearchCV(
    estimator=MLPRegressor(random_state=42), 
    param_grid=param_grid, 
    cv=cv,  
    scoring='neg_mean_absolute_percentage_error',  
    n_jobs=-1,  
    verbose=2  
)


In [221]:
# Блок 6: Обучение GridSearchCV
print("Начинается подбор гиперпараметров...")
grid_search.fit(X_train_scaled, y_train)  # поиск модели

Начинается подбор гиперпараметров...
Fitting 5 folds for each of 4374 candidates, totalling 21870 fits


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=42, test_size=0.1, train_size=None),
             estimator=MLPRegressor(random_state=42), n_jobs=-1,
             param_grid={'activation': ['logistic'], 'alpha': [0.92, 0.96, 1.0],
                         'beta_1': [0.8, 0.85, 0.9],
                         'beta_2': [0.9, 0.95, 0.99], 'early_stopping': [True],
                         'epsilon': [1e-06],
                         'hidden_layer_sizes': [(8,), (10,), (12,)],
                         'learning_rate_init': [0.03, 0.04, 0.05],
                         'max_iter': [200], 'n_iter_no_change': [5, 10],
                         'solver': ['adam'], 'tol': [0.01, 0.05, 0.1],
                         'validation_fraction': [0.1, 0.2, 0.3]},
             scoring='neg_mean_absolute_percentage_error', verbose=2)

In [222]:
# Блок 7: Лучшая модель
best_params = grid_search.best_params_  # Получение лучших гиперпараметров
print("Лучшие параметры:", grid_search.best_params_)
print("Значение на кросс-валидации:", -grid_search.best_score_ * 100, "%")

# Получение модели с лучшими параметрами
best_model = grid_search.best_estimator_

# Прогнозирование на тестовом наборе на ЛУЧШЕЙ МОДЕЛИ
y_pred_best = best_model.predict(X_test_scaled)

Лучшие параметры: {'activation': 'logistic', 'alpha': 1.0, 'beta_1': 0.85, 'beta_2': 0.95, 'early_stopping': True, 'epsilon': 1e-06, 'hidden_layer_sizes': (10,), 'learning_rate_init': 0.05, 'max_iter': 200, 'n_iter_no_change': 5, 'solver': 'adam', 'tol': 0.01, 'validation_fraction': 0.3}
Значение на кросс-валидации: 8.908535789975465 %


In [223]:
# Блок 8: Вычисление метрик
# Вычисление MAPE (Mean Absolute Percentage Error) для оценки точности
mape_best = mean_absolute_percentage_error(y_test, y_pred_best) * 100
# Вычисление RMSE (Root Mean Squared Error) для оценки отклонений предсказаний от реальных значений
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

# Вывод лучших метрик
print(f"MAPE: {mape_best:.2f}%")
print(f"RMSE: {rmse_best:.2f}")

MAPE: 8.92%
RMSE: 3.24


In [224]:
# Блок 9: Создание DataFrame для тестовых значений
# Сравнение истинных значений с предсказанными для тестового
test_comparison_model = pd.DataFrame({"yn": y_test.values, "y_pred_best": y_pred_best})
print(test_comparison_model)

    yn  y_pred_best
0   19    18.561763
1   32    24.975439
2   25    25.451684
3   27    25.735553
4   22    22.908958
5   22    22.373955
6   28    21.080643
7   24    23.889509
8   21    24.987863
9   25    25.157197
10  29    24.453451
11  27    26.510208
12  26    22.492444
13  26    27.323695
14  29    26.927798
15  23    19.745215
16  27    24.953920
17  30    25.708617


In [225]:
# Блок 10: Сохранение результатов
# Сохранение результатов и метрик в Excel файл
test_comparison_best = pd.DataFrame({"yn": y_test.values, "y_pred_py": y_pred_best})
metrics_best = pd.DataFrame({"Метрика": ["MAPE", "RMSE"], "Значение": [mape_best, rmse_best]})

# Запись в Excel
output_file = 'models_results_optimized.xlsx'  # Имя выходного файла
with pd.ExcelWriter(output_file) as writer:
    test_comparison_best.to_excel(writer, sheet_name='Результаты лучшей модели', index=False)  # Результаты модели
    metrics_best.to_excel(writer, sheet_name='Метрики лучшей модели', index=False)  # Метрики модели